# Data Preparation

## Introduction

The first part of building any AI model is finding appropriate data. For your project, the model will be classifying images, so your data should be images as well. In this module, you will learn how to obtain your data using different methods and process it, so it is ready to be ingested by the model.

## Image Representation

Most of the digital images available today are represented in __raster__ format. Raster image contains a two-dimensional table of _pixels_ - small squared (or rectangular) shape elements of the image.

![](https://drive.google.com/uc?export=view&id=1AeC4a6YCNDBFm3ArdyiKgM7rCHn92xg1)

There are various ways the information can be embedded into pixel, however we will use the most popular method of RGB, representing three-sized tuple of Red, Green, and Blue intensities.


![](https://drive.google.com/uc?export=view&id=1yjsqrUxGGecJXvrwFHSX6ZJ1jrjUK11R)

In other words, every pixel is a sequence of three numbers (usually from 0 to 255) and image is a table of pixels.

![](https://drive.google.com/uc?export=view&id=1Hi3HFJU-yXJX1vtd7lyhIBtSY5WEeXCW)

Hence, input representation for any colored image can be thought of as three-dimensional table, with dimensions ($width$, $height$, $3$), where ($width$, $height$) tuple is known as _image resolution_. The ratio of the two ($width/frac$) is known as aspect ratio.

## Obtaining Data

There are various ways the data can be obtained for the project, however we will be focusing on two methods:

1. Creating images using photo camera (such as the one you have in your phones).
2. Creating snapshots from a video feed (which can be created using web camera).

These methods have no meaningful advantages over each other for this project. You can use whichever is more preferrable to you or combination of both. Beloww is the sceleton code that you can use to prepare the data for data pre-processing.

Mount your Google Drive using your Gmail account. Proceed to the folder where you will store the data for your project.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
### Name Project folder
proj_name = 'rps/train'
###

proj_path = '/content/gdrive/MyDrive/'+proj_name

You only need to run following cells once to create folders. Cells output will throw an error for any subsequent runs.

In [ ]:
%mkdir $proj_path

In [ ]:
%mkdir {proj_path+'/rock'}

In [ ]:
%mkdir {proj_path+'/paper'}

In [ ]:
%mkdir {proj_path+'/scissors'}

Change current directory to project folder.

In [3]:
## Change the current directory
%cd $proj_path
%ls

/content/gdrive/MyDrive/rps/train
paper/  rock/  scissors/


### Using Web Camera

Run the following cell. It will create a function that will take a snapshot from your webcam by pressing a button and save the file in your current folder.

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

Change the parameters in the cells to choose number of snapshots for each gesture. Run the cell and follow the prompt.

![](https://drive.google.com/uc?export=view&id=1YrurRp9FYI3yGmzqqIG9wuPrIpBiAO1D)

In [ ]:
from IPython.display import Image

### Choose these parameters
# maxRock - number of images taken with "rock" gesture
# maxPaper - number of images taken with "paper" gesture
# maxScissors - number of images taken with "scissors" gesture
###

maxRock = 5
maxPaper = 5
maxScissors = 5

### End initialization
###
try:
  ct = int(1)
  maxPaper = maxRock + maxPaper
  maxScissors = maxPaper + maxScissors
  while True:
    if ct <= maxRock:
      print("Show Rock and press Capture!")
      label = "rock"
    elif ct <= maxPaper:
      print("Show Paper and press Capture!")
      label = "paper"
    else:
      print("Show Scissors and press Capture!")
      label = "scissors"

    filename = take_photo(label + "/" +label+'_'+str(ct))

    print('Saved to {}'.format(filename))

    ct += 1
    if ct > maxScissors:
      break
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

Show Rock and press Capture!
name 'take_photo' is not defined


### Using photo camera

0. If you have iPhone, please change File formats before you take pictures. In Settings -> Camera -> Formats, choose Most Compatible. You can change it back after pictures are taken.
1. Make pictures of different gestures using you camera.
2. Download and Install Google Drive App (if not already installed).
3. Copy images in the respective folders.
4. Run the cell below to rename files to general convention.

## Creating Data Tips

1. Ensure image is of high quality and the gesture is discernible by human eye. If a human cannot classify the image, neither will machine!
2. Keep background clean and the classification object large enough to improve quality of learning.
  - If there are extraneous items in the background, the algorithm may incorrectly aim these items to be part of classification solution.
  - If object is too small and not prominent, algorithm will tend to focus on background more, reducing accuracy.
3. Ensure appropriate aspect ratio. Changing aspect ratio will stretch/shrink object on the image, which makes learning more difficult.
4. Ensure labels are correct, i.e. appropriate images are stored in their respective folders.

## Data pre-processing

To uphold principle of fairness and simplify machine learning process, all images need to be converted into same file format (.jpg) and same resolution. Please run the cell below to convert the files to common format.

Participants are also encouraged to manually edit the pictures, using tips above, to improve accuracy of their learning algorithm.

Converts code and renames bad files with starting 'z' name.

In [28]:
from os import listdir, rename
from os.path import isfile, join
from PIL import Image
from os.path import splitext

proj_path = '/content/gdrive/MyDrive/rps/train'


for label in ['rock', 'paper', 'scissors']:
    for i in [f for f in listdir(join(proj_path, label)) if isfile(join(proj_path, label, f))]:
        file_path = join(proj_path, label, i)
        file_name, file_ext = splitext(i)

        try:
            im = Image.open(file_path)
            im = im.resize((640, 480))

            if file_ext.lower() in ['.png', '.jpeg']:
                # Convert RGBA to RGB if the image has an alpha channel
                if im.mode == 'RGBA':
                    im = im.convert('RGB')
                new_file_path = join(proj_path, label, f"{file_name}.jpg")
                im.save(new_file_path, 'jpeg')
                print(f"Successfully Converted {i} to {file_name}.jpg")
                # Optionally, remove the original file
                # os.remove(file_path)
            else:
                new_file_path = join(proj_path, label, f"{file_name}.jpg")
                im.save(new_file_path, 'jpeg')
                print(f"Successfully Resized {i} and saved as {file_name}.jpg")

        except Exception as e:
            print(f"Error processing file {i} at {file_path}: {e}")
            # Rename the problematic file with 'z_' prefix
            new_file_path = join(proj_path, label, f"z_{i}")
            rename(file_path, new_file_path)
            print(f"Renamed problematic file {i} to z_{i}")



Successfully Resized rock_11.jpg and saved as rock_11.jpg
Successfully Resized rock_7.jpg and saved as rock_7.jpg
Successfully Resized rock_13.jpg and saved as rock_13.jpg
Successfully Resized rock_17.jpg and saved as rock_17.jpg
Successfully Resized rock_39.jpg and saved as rock_39.jpg
Successfully Resized rock_19.jpg and saved as rock_19.jpg
Successfully Resized rock_21.jpg and saved as rock_21.jpg
Successfully Resized rock_23.jpg and saved as rock_23.jpg
Successfully Resized rock_37.jpg and saved as rock_37.jpg
Successfully Resized rock_27.jpg and saved as rock_27.jpg
Successfully Resized rock_35.jpg and saved as rock_35.jpg
Successfully Resized rock_29.jpg and saved as rock_29.jpg
Successfully Resized rock_31.jpg and saved as rock_31.jpg
Successfully Resized rock_25.jpg and saved as rock_25.jpg
Successfully Resized rock_33.jpg and saved as rock_33.jpg
Successfully Resized rock_20.jpg and saved as rock_20.jpg
Successfully Resized rock_1.jpg and saved as rock_1.jpg
Successfully Resiz

ONLY AFTER REVIEWING AND DELETING FILES - Then RENAME, REORDER remaining files to jpg - Wait 1-2 minutes between running this and prior section to be sure the Z files are no longer in folder.

In [29]:
import os

def renumber_images(base_path):
    categories = ['rock', 'paper', 'scissors']
    for category in categories:
        # Renumber images in the train folder
        train_path = os.path.join(base_path, 'train', category)
        renumber_folder(train_path, category)

        # Renumber images in the test folder
        test_path = os.path.join(base_path, 'test', category)
        renumber_folder(test_path, category)

def renumber_folder(folder_path, class_name):
    if not os.path.exists(folder_path):
        print(f"Folder not found: {folder_path}")
        return

    files = os.listdir(folder_path)
    files.sort()  # Sort to maintain a consistent order

    # First, rename all files to a temporary name
    temp_files = []
    for idx, filename in enumerate(files, start=1):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            temp_filename = f"temp_{class_name}_{idx:05d}.jpg"
            temp_file_path = os.path.join(folder_path, temp_filename)
            os.rename(file_path, temp_file_path)
            temp_files.append(temp_filename)

    # Then, rename all temporary files to the final name
    for idx, temp_filename in enumerate(temp_files, start=1):
        temp_file_path = os.path.join(folder_path, temp_filename)
        if os.path.isfile(temp_file_path):
            new_filename = f"{class_name}_{idx}.jpg"
            new_file_path = os.path.join(folder_path, new_filename)
            os.rename(temp_file_path, new_file_path)
            print(f"Renamed {temp_file_path} to {new_file_path}")

# Path to your dataset
base_path = '/content/gdrive/My Drive/rps'
renumber_images(base_path)

Renamed /content/gdrive/My Drive/rps/train/rock/temp_rock_00001.jpg to /content/gdrive/My Drive/rps/train/rock/rock_1.jpg
Renamed /content/gdrive/My Drive/rps/train/rock/temp_rock_00002.jpg to /content/gdrive/My Drive/rps/train/rock/rock_2.jpg
Renamed /content/gdrive/My Drive/rps/train/rock/temp_rock_00003.jpg to /content/gdrive/My Drive/rps/train/rock/rock_3.jpg
Renamed /content/gdrive/My Drive/rps/train/rock/temp_rock_00004.jpg to /content/gdrive/My Drive/rps/train/rock/rock_4.jpg
Renamed /content/gdrive/My Drive/rps/train/rock/temp_rock_00005.jpg to /content/gdrive/My Drive/rps/train/rock/rock_5.jpg
Renamed /content/gdrive/My Drive/rps/train/rock/temp_rock_00006.jpg to /content/gdrive/My Drive/rps/train/rock/rock_6.jpg
Renamed /content/gdrive/My Drive/rps/train/rock/temp_rock_00007.jpg to /content/gdrive/My Drive/rps/train/rock/rock_7.jpg
Renamed /content/gdrive/My Drive/rps/train/rock/temp_rock_00008.jpg to /content/gdrive/My Drive/rps/train/rock/rock_8.jpg
Renamed /content/gdrive/